In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, BatchNormalization, LeakyReLU, Conv2DTranspose, Conv2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
# import cv2
import os
import time
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


# Download the dataset

In [19]:
# # Load the CelebA dataset and split into training and validation sets
# data, info = tfds.load('celeb_a', split=['train[:80%]', 'train[80%:]', 'test'], with_info=True)

# # Define the image size and batch size
# img_size = 64
# batch_size = 32

# # Define a function to preprocess the images
# def preprocess_image(sample):
#     image = sample['image']
#     image = tf.image.resize(image, size=(img_size, img_size))
#     image = tf.cast(image, dtype=tf.float32)
#     image = (image / 127.5) - 1.0  # Normalize the pixel values to the range [-1, 1]
#     return image

# # Apply the preprocessing function to the training and validation datasets
# train_data = data[0].map(preprocess_image).shuffle(buffer_size=10000).batch(batch_size)
# val_data = data[1].map(preprocess_image).batch(batch_size)

# # Apply the preprocessing function to the test dataset
# test_data = data[2].map(preprocess_image).batch(batch_size)
dataset = tf.keras.utils.image_dataset_from_directory(
    "celeba_gan", label_mode=None, image_size=(64, 64), batch_size=32
)
dataset = dataset.map(lambda x: x / 255.0)

Found 202599 files belonging to 1 classes.


In [21]:
for x in dataset:
    plt.axis("off")
    plt.imshow((x.numpy() * 255).astype("int32")[0])
    break

# Prepare the dataset
train_dataset = dataset.shuffle(1000).batch(batch_size)


In [22]:

# fig, axs = plt.subplots(4, 8, figsize=(16, 8))
# for i, sample in enumerate(train_data.take(32)):
#     row = i // 8
#     col = i % 8
#     axs[row, col].imshow((sample.numpy()[0] + 1.0) / 2.0)
#     axs[row, col].axis('off')
# plt.show()
# print("testing Images")
# # Visualize some images from the test dataset
# fig, axs = plt.subplots(4, 8, figsize=(16, 8))
# for i, sample in enumerate(test_data.take(32)):
#     row = i // 8
#     col = i % 8
#     axs[row, col].imshow((sample.numpy()[0] + 1.0) / 2.0)
#     axs[row, col].axis('off')
# plt.show()

# Define the Model

In [23]:

discriminator = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(64, 64, 3)),
        tf.keras.layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
# discriminator.summary()
latent_dim = 128

generator = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(latent_dim,)),
        tf.keras.layers.Dense(8 * 8 * 128),
        tf.keras.layers.Reshape((8, 8, 128)),
        tf.keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)
# generator.summary()
def make_dcgan_model(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential([generator, discriminator])
    return model

dcgan = make_dcgan_model(generator, discriminator)


# Train the model 

In [24]:
# 1. Define the loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss


# 2. Define the optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [25]:
# 3. Create a training step function
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [26]:
# 4. Create a training loop
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        print(f'Time for epoch {epoch + 1} is {time.time() - start} sec')


In [27]:
# Train the model
train(train_data, epochs=2)

ValueError: in user code:

    /tmp/ipykernel_631162/1702880358.py:7 train_step  *
        generated_images = generator(noise, training=True)
    /home/cse479/abiswas3/.conda/envs/tensorflow-env/lib/python3.9/site-packages/keras/engine/base_layer.py:1020 __call__  **
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    /home/cse479/abiswas3/.conda/envs/tensorflow-env/lib/python3.9/site-packages/keras/engine/input_spec.py:250 assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer generator is incompatible with the layer: expected axis -1 of input shape to have value 128 but received input with shape (32, 100)


In [5]:
# Define the loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# Define the optimizer
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Define the checkpoint directory and checkpoint saver
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# Define the training loop
@tf.function
def train_step(images):
    noise = tf.random.normal([32, 100])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

def train(dataset, epochs):
    # Create lists to store losses for plotting
    gen_losses = []
    disc_losses = []
    
    # Define early stopping callback
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)

    for epoch in range(epochs):
        gen_loss_avg = tf.keras.metrics.Mean()
        disc_loss_avg = tf.keras.metrics.Mean()

        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)
            gen_loss_avg.update_state(gen_loss)
            disc_loss_avg.update_state(disc_loss)

        # Save losses for plotting
        gen_losses.append(gen_loss_avg.result())
        disc_losses.append(disc_loss_avg.result())

        # Save a checkpoint every 5 epochs
        if (epoch + 1) % 5 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

        # Print progress
        print(f'Epoch {epoch + 1}, gen loss={gen_loss_avg.result()}, disc loss={disc_loss_avg.result()}')

    # Save final checkpoint
    checkpoint.save(file_prefix=checkpoint_prefix)

    # Return losses for plotting
    return gen_losses, disc_losses

# Train the model
EPOCHS = 50
gen_losses, disc_losses = train(train_data, EPOCHS)

# Plot the generator and discriminator losses
plt.plot(gen_losses, label='Generator loss')
plt.plot(disc_losses, label='Discriminator loss')
plt.legend()
plt.show()


ValueError: in user code:

    /tmp/ipykernel_629036/2169084504.py:30 train_step  *
        real_output = discriminator(images, training=True)
    /home/cse479/abiswas3/.conda/envs/tensorflow-env/lib/python3.9/site-packages/keras/engine/base_layer.py:1037 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    /home/cse479/abiswas3/.conda/envs/tensorflow-env/lib/python3.9/site-packages/keras/engine/sequential.py:369 call
        return super(Sequential, self).call(inputs, training=training, mask=mask)
    /home/cse479/abiswas3/.conda/envs/tensorflow-env/lib/python3.9/site-packages/keras/engine/functional.py:414 call
        return self._run_internal_graph(
    /home/cse479/abiswas3/.conda/envs/tensorflow-env/lib/python3.9/site-packages/keras/engine/functional.py:550 _run_internal_graph
        outputs = node.layer(*args, **kwargs)
    /home/cse479/abiswas3/.conda/envs/tensorflow-env/lib/python3.9/site-packages/keras/engine/base_layer.py:1020 __call__
        input_spec.asse